# High-Level Synthesis Introduction

----------------------------------------------------

## Chapter1.	Tutorial Description
    Requirement & Lab files
 
## Chapter2.	High-Level Synthesis Introduction
     Tutorial Design Description
     Lab1.	HLS Project 생성해보기
     Lab2.	Tcl Command Interface 사용해보기
     Lab3.	설계 최적화를 위해 Solution 사용해보기
     Conclusion
	
----------------------------------------------------	

## Tutorial Design Description
## Lab1.	HLS Project 생성해보기

	+ Step1.	Creating a New Project
		(1) 새로운 프로젝트를 생성한다.
			- 작업영역 지정
			- Project 이름지정
			- C file 추가 ( 여러개여도 하나의 Top Function지정해 줘야 함 )
			- C testbench 파일 추가
			- 헤더파일 위치에 따른 CFLAGS 지정 ( 추후 더 찾아봐야. 이해잘안됨 )
			- Solution Configuration (  Clock, Clock Uncertainty, Part )
			- 추가된 예제
				* fir.c			실제 동작이 기술된 C source 
				* fir_test.c	위 C source를 포함한 testbench
				* out.gold.dat	동작 결과로 예측되는 결과(기대값)를 담은 데이터 결과 파일
				
	+ Step2.	Validate the C Source Code
		(1) Run C Simulation ( Project > Run C Simulation )
			- C design의 compile & execute
			- testbench의 main()함수는 실제 설계대상인 C design의 Top function을 포함
			- 컴파일 ( Run C Simulation ) 을 하면, testbench는 Top function으로부터 결과값 데이터를 out.dat로 저장
			- 컴파일 수행 결과인 out.dat와 미리 준비된 out.gold.dat가 같으면 0, 다르면 1을 출력
			- 1이 출력되거나, 리턴값이 없는 경우, Debug.
			- C design의 source code검증 후, 이 소스를 RTL design으로 Synthesis ( Step3 )
			
	+ Step3.	High-Level Synthesis
		(1) Run C Synthesis ( Solution > Run C Synthesis > Active Solution )
			- C Synthesis 결과 Report 분석
				* Performance Estimates
				* Utilization Estimates
				* Interface Report
				
	+ Step4.	RTL Verification
		- HLS에서는 RTL 검증에 C design 검증용으로 사용한 testbench를 재활용할수 있음
		(1) Run C/RTL CoSimulation ( Solution > Run C/RTL CoSimulation )
		
	+ Step5.	IP Creation
		(1) Export RTL ( Solution > Export RTL )
		(2) IP Catalog > Click OK
		
		
		
	+ Summary :
		- C design -> C testbench의 main() 안에 C design의 top function을 넣고 C testbench작성
		- C design의 testbench 결과벡터를 작성 out.gold.dat
		- [ Run C Simulation ]
			* 작성된 C testbench를 통해 C Simulation 수행
			* Top C function 동작 결과값 데이터를 out.dat로 저장
			* 기대값 out.gold.dat 과 Simulation 수행 결과 out.dat 비교 ( 정상 시 "0" 반환 )

## Lab2.	Tcl Command Interface 사용해보기
	
	+ Step1.	Creating a Tcl file
		(1) Vivado HLS 2019.2 Command Prompt 를 열고 예제의 Introduction 위치로 이동
		(2) Lab1 프로젝트 폴더에서 기 생성된 script.tcl을 Lab2로 복사
			* cp lab1\fir_prj\solution\script.tcl lab2\run_hls.tcl
		(3) cd lab2 로 위치 이동
		(4) run_hls.tcl 파일을 열고, open_project와 open_solution 명령에 -reset추가
		(5) 파일 끝줄에 exit 명령 추가 ---> 파일 저장 후 종료
		(6) $ vivado_hls -f run_hls/tcl
		(7) 위 명령 실행으로, syn, sim, impl 폴더 아래 각각 생성된 결과 확인
			* syn > report	: synthesis report
			* sim > report	: simulation result
			* impl > ip		: output package
			* impl			: final RTL ( Verilog or VHDL )
			

## Lab3.	설계 최적화를 위해 Solution 사용해보기

	+ Step1.	Creating a New Project
		(1) Vivado HLS 2019.2 Command Prompt 를 열고 예제의 Introduction\lab3 위치로 이동
		(2) $ vivado_hls -f run_hls.tcl
		(3) $ vivado_hls -p fir_prj 
	
	+ Step2.	Optimization the I/O Interface
		- Post C must have a single port RAM access
		- Post X must have an input data valid signal
		(1) 새로운 solution을 default 조건으로 생성한다. ( Project > New Solution ) 
		(2) Directive tab에서 최적화를 수행할 변수/port를 선택한 다음 optimization directives를 적용한다.
			* 방법 : Directive tab에서 최적화를 수행할 signal/port 선택 후 우클릭 > Insert Directive 
					Directive, Destination, mode등을 선택한 후 적용
		(3) Run C Synthesis ( ---> "Yes" )
		(4) Outline 탭에서 결과 확인
		
	+ Step3.	Analyze the Results
		(1) Analysis perspective 로 이동, 리포트 내용 확인
			* Schedule Viewer (top row lists : control states ( RTL FSM과 밀접한 상관을 가지며 1:1매핑 안됨 )
			* Resource Viewer (추가 내용확인이 필요)
			
	+ Step4.	Optimize for the Highest Throughput ( Lowest Interval )		
		(1) C 코드상에서의 for loop
			* default : 각 iteration은 하나의 loop body를 합성, 재활용하여 순차 진행 되는 형태로 구성됨
			* 연산을 parallel로 수행하기 위해 for loop를 풀어낼 수 있음 ( unroll the loop )
		(2) C 코드상에 array 변수	: default Block Ram, shift_reg로 설정 가능
		
		(3) 다음과 같이 Synthesis perspective를 눌러 윈도우를 전환한 다음..
			- New Solution (solution3)을 default로 생성
			- Directive 탭에서 loop, shift_reg 변수를 선택 후 우클릭하여 각각 Insert Directive를 해준다.
			- unroll, shift_reg 등 Directive drop-down 메뉴에서 선택하여 추가해준다.
			- Explorer창에서 solution3의 constraints > directives.tcl 확인 후 
		(4) Synthesis ( C Synthesis 를 선택해준다 ) 를 해 주고
			(5) Project > Compare Report 를 선택한 다음, 모든 solution을 추가하여 OK